## 1. Initialize project
Create or use an existing project to scope the RAG scenario

In [1]:
import  digitalhub as dh

project = dh.get_or_create_project("rag-kubeai")

/home/raman/python3.10/lib/python3.10/site-packages/digitalhub/stores/client/dhcore/configurator.py:96: UserWarning: Backend API level is higher than library version. You should consider updating the library.
  warn("Backend API level is higher than library version. You should consider updating the library.")


## 2. Define and Deploy Training Function
Training function is a python job that performs LLM fine tuning from a predefined HuggingFace model and dataset.

In [2]:
func = project.new_function(
    name="train-llm", 
    kind="python", 
    python_version="PYTHON3_10", 
    code_src="src/llama_sft_training.py",  
    handler="train_and_log_model",
    requirements=["pandas==2.1.4", "tqdm==4.66.5", "openai==1.8.0", "spacy==3.7.5", "torch==2.5.1", "llama-index==0.9.33", "huggingface-hub==0.27.1", "rank-bm25==0.2.2", "sentence-transformers==4.1.0", "ranx==0.3.20", "transformers==4.48.0", "openpyxl==3.1.5", "rouge-score==0.1.2", "FlagEmbedding==1.3.4", "wandb==0.19.11", "nltk==3.8.1", "trl==0.13", "bitsandbytes==0.45.5", "datasets==3.6.0"]
)

## 3. Perform Training
Training parameters define the model and dataset to be used. Training run refers to HF_TOKEN secret and expects a volume to be created of an appropriate size.

In [3]:
train_run = func.run(action="job",
                     profile="1xa100",
                     parameters={
                         "model_id": "meta-llama/Llama-3.1-8B",
                         "model_name": "llmpa-tracked",
                         "hf_dataset_name": "team-bay/data-science-qa",
                         "train_data_path": "DataScienceDataset.csv",
                         "dev_data_path": "DataScienceDataset.csv",
                     },
                     secrets=["HF_TOKEN"],
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-llmpa",
                        "mount_path": "/app/local_data",
                        "spec": { "size": "20Gi" }}]
					)